![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)

## Deploy SQL Server 2019 big data cluster on an existing Kubernetes cluster

This notebook walks through the process of deploying a SQL Server 2019 big data cluster on an existing Kubernetes cluster

* Follow the instructions in the **Dependencies** cell to install the dependencies.
* Make sure you have the target cluster set as the current context in your kubectl config file.
* The **Required information** cell will prompt you for the required information to create a SQL Server 2019 big data cluster.



### **Dependencies**
<table>
<colgroup>
<col style="width: 10%" />
<col style="width: 85%" />
<col style="width: 10%" />
</colgroup>
<thead>
<tr class="header">
<th>Tool</th>
<th>Description</th>
<th>Installation</th>
</tr>
</thead>
<tbody>
<td><strong>kubectl</strong></td>
<td>Command-line tool for monitoring the underlying Kuberentes cluster (<a href="https://kubernetes.io/docs/tasks/tools/install-kubectl/">More info</a>)</td>
<td><a href="https://kubernetes.io/docs/tasks/tools/install-kubectl/#install-kubectl-binary-using-native-package-management">Install</a></td>
</tr>
<tr>
<td><strong>mssqlctl</strong></td>
<td>Command-line tool for installing and managing a big data cluster</td>
<td><a href="https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-mssqlctl?view=sqlallproducts-allversions">Install</a></td>
</tr>
</tbody>
</table>
<p>

### **Check dependencies**

In [1]:
#Run command helper function
def run_command():
    print("Executing: " + cmd)
    !{cmd}
    if _exit_code != 0:
        raise SystemExit(f'Shell command:\n\n\t{cmd}\n\nreturned non-zero exit code: ' + str(_exit_code) + '.\n')
    print(f'Successfully executed: {cmd}')

cmd = 'kubectl version --client=true'
run_command()
cmd = 'mssqlctl --version'
run_command()

### **Show current context**

In [2]:
cmd = ' kubectl config current-context'
run_command()

### **Required information**

In [3]:
import getpass
def get_user_input(input_name, is_password = False, confirm_password = False):
    if is_password:
        user_input = getpass.getpass(prompt = input_name)
        if confirm_password:
            user_input_confirm = getpass.getpass(prompt = 'Confirm '+ input_name)
            if user_input != user_input_confirm:
                raise SystemExit(f'{input_name} does not match the confirmation password')
        print(f'{input_name}: ******')
    else:
        user_input = input(input_name)
        print(input_name + ': ' + user_input)
    if user_input == "":
        raise SystemExit(f'{input_name} is required')
    return user_input;

mssql_cluster_name = get_user_input('Cluster name')
bdc_controller_username = get_user_input('Controller username')
bdc_password = get_user_input('Controller password', True, True)
print('Knox and SQL Server will use the same password')
docker_username = get_user_input('Docker username')
docker_password = get_user_input('Docker password', True)

### **Default settings**

In [4]:
mssqlctl_configuration_profile = 'aks-dev-test'
mssqlctl_configuration_file = 'mssql-bdc-configuration'

# Show parameter values
print('')
print(f'mssql_cluster_name = {mssql_cluster_name}')
print(f'docker_username = {docker_username}')
print(f'mssqlctl_configuration_profile = {mssqlctl_configuration_profile}')
print(f'mssqlctl_configuration_file = {mssqlctl_configuration_file}')
print(f'bdc_controller_username = {bdc_controller_username}')
print('')

### **List the MSSQLCTL configuration profiles**

In [5]:
import os
os.environ["ACCEPT_EULA"] = 'yes'
cmd = f'mssqlctl bdc config list'
run_command()

### **Create a MSSQLCTL configuration file**

In [6]:
# Create a configuration file
cmd = f'mssqlctl bdc config init --source {mssqlctl_configuration_profile} --target {mssqlctl_configuration_file} --force'
run_command()

# Set the 'big data cluster' name
jsonPath = '"metadata.name=''{0}''"'.format(mssql_cluster_name)
cmd = f'mssqlctl bdc config section set -c {mssqlctl_configuration_file} -j {jsonPath}'
run_command()

### **Create SQL Server 2019 big data cluster**

In [7]:
import os
print (f'Creating SQL Server 2019 big data cluster: {mssql_cluster_name} using configuration file {mssqlctl_configuration_file}')
os.environ["CONTROLLER_USERNAME"] = bdc_controller_username
os.environ["CONTROLLER_PASSWORD"] = bdc_password
os.environ["MSSQL_SA_PASSWORD"] = bdc_password
os.environ["KNOX_PASSWORD"] = bdc_password
os.environ["DOCKER_USERNAME"] = docker_username
os.environ["DOCKER_PASSWORD"] = docker_password

cmd = f'mssqlctl bdc create -c {mssqlctl_configuration_file} --accept-eula yes'
run_command()

### **Login to SQL Server 2019 big data cluster**

In [8]:
cmd = f'mssqlctl login --cluster-name {mssql_cluster_name}'
run_command()

### **Show SQL Server 2019 big data cluster endpoints**

In [9]:
import json
import pandas as pd
from IPython.display import *
pd.set_option('display.max_colwidth', -1)

def formatColumnNames(column):
    return ' '.join(word[0].upper() + word[1:] for word in column.split())

def show_results(results):
    df = pd.DataFrame(results)
    df.columns = [formatColumnNames(n) for n in results[0].keys()]
    mydata = HTML(df.to_html(render_links=True))
    display(mydata)

cmd = f'mssqlctl bdc endpoint list'
endpointsResults  = !{cmd}
endpointsInJson = json.loads(''.join(endpointsResults))
show_results(endpointsInJson)

### **Connect to master SQL Server instance in Azure Data Studio**
Click the link below to connect to the master SQL Server instance of the SQL Server 2019 big data cluster.

In [10]:
filteredEndpoints = [x for x in endpointsInJson if x['name'] == 'sql-server-master']
if filteredEndpoints and len(filteredEndpoints) == 1:
    display(HTML("<h3><a href=\"command:azdata.connect?{&quot;serverName&quot;:&quot;"+filteredEndpoints[0]['endpoint']+"&quot;,&quot;providerName&quot;:&quot;MSSQL&quot;, &quot;authenticationType&quot;:&quot;SqlLogin&quot;,&quot;userName&quot;:&quot;sa&quot;,&quot;password&quot;:&quot;"+bdc_password+"&quot;}\">Click here to connect to SQL Server</a></h3>"))